### 3-3．모델작성
#### 데이터의 리드와 확인

In [1]:
# Pandas의 로드
import pandas as pd

# 파일 리드
bank_df_new = pd.read_csv('data/bank-prep.csv', sep=',')

# 선두에서 5행까지 표시
bank_df_new.head()

,age,default,balance,housing,loan,day,duration,campaign,pdays,previous,...,dec,feb,jan,jul,jun,mar,may,nov,oct,sep
0,58,0,2143,1,0,5,261,1,-1,0,...,0,0,0,0,0,0,1,0,0,0
1,36,0,265,1,1,5,348,1,-1,0,...,0,0,0,0,0,0,1,0,0,0
2,25,0,-7,1,0,5,365,1,-1,0,...,0,0,0,0,0,0,1,0,0,0
3,53,0,-3,0,0,5,1666,1,-1,0,...,0,0,0,0,0,0,1,0,0,0
4,24,0,-103,1,1,5,145,1,-1,0,...,0,0,0,0,0,0,1,0,0,0


#### 불균형데이터의 균형화

In [2]:
!pip show imbalanced-learn

Name: imbalanced-learn
Version: 0.10.1
Summary: Toolbox for imbalanced dataset in machine learning.
Home-page: https://github.com/scikit-learn-contrib/imbalanced-learn
Author: 
Author-email: 
License: MIT
Location: c:\users\admin\anaconda3\lib\site-packages
Requires: joblib, numpy, scikit-learn, scipy, threadpoolctl
Required-by: 


In [3]:
!pip list

Package                       Version
----------------------------- --------------------
absl-py                       1.4.0
alabaster                     0.7.12
anaconda-client               1.11.0
anaconda-navigator            2.4.0
anaconda-project              0.11.1
anyio                         3.5.0
appdirs                       1.4.4
argon2-cffi                   21.3.0
argon2-cffi-bindings          21.2.0
arrow                         1.2.2
astroid                       2.11.7
astropy                       5.1
astunparse                    1.6.3
atomicwrites                  1.4.0
attrs                         21.4.0
Automat                       20.2.0
autopep8                      1.6.0
Babel                         2.9.1
backcall                      0.2.0
backports.functools-lru-cache 1.6.4
backports.tempfile            1.0
backports.weakref             1.0.post1
bcrypt                        3.2.0
beautifulsoup4                4.11.1
binaryornot                   0.4.4
bi

In [4]:
!pip install imbalanced-learn

In [4]:
# Numpy와 RandomUnderSampler의 로드
import numpy as np
from imblearn.under_sampling import RandomUnderSampler

# 데이터 세트를 설명변수와 목적변수로 분할
X = np.array(bank_df_new.drop('y', axis=1))
Y = np.array(bank_df_new['y'])

# y가 1과 0인 데이터의 건수를 카운트
print(np.sum(Y == 1), np.sum(Y == 0))

# y가 1인 데이터의 건수에 맞춰서 다운사이징
sampler = RandomUnderSampler(random_state=42)
X, Y = sampler.fit_resample(X, Y)

# y가 1과 0인 데이터 건수를 카운트
print(np.sum(Y == 1), np.sum(Y == 0))

820 6113
820 820


In [5]:
# Numpy와 RandomOverSampler의 로드
import numpy as np
from imblearn.over_sampling import RandomOverSampler

# 데이터 세트를 설명변수와 목적변수로 분할
X = np.array(bank_df_new.drop('y', axis=1))
Y = np.array(bank_df_new['y'])

# y가 1과 0인 데이터의 건수를 카운트
print(np.sum(Y == 1), np.sum(Y == 0))

# y가 1인 데이터의 건수에 맞춰서 다운사이징
sampler = RandomOverSampler(random_state=42)
X, Y = sampler.fit_resample(X, Y)

# y가 1과 0인 데이터 건수를 카운트
print(np.sum(Y == 1), np.sum(Y == 0))

820 6113
6113 6113


#### 결정트리의 작성과 검증

In [6]:
# Scikit-learn의 로드
from sklearn.model_selection import KFold
from sklearn import tree
from sklearn.metrics import accuracy_score

# K-Fold 교차검증
kf = KFold(n_splits=10, shuffle=True)
# 모델의 정밀도를 보존할 준비
scores = []
# 데이터를 셔플하고, 훈련데이터와 테스트데이트로 분할
for train_id, test_id in kf.split(X):
    # 훈련 데이터를 사용하여 모델을 작성
    x = X[train_id]
    y = Y[train_id]
    clf = tree.DecisionTreeClassifier()
    clf.fit(x,y)
    # 테스트 데이터에 모델을 적용
    pred_y = clf.predict(X[test_id])
    # 모델의 정밀도를 계산해서 보존
    score = accuracy_score(Y[test_id], pred_y)
    scores.append(score)

# 모델의 평균정밀도, 표준편차를 확인
scores = np.array(scores)
print(scores.mean(), scores.std())

0.9626208258782502 0.00557025094096199


In [7]:
# Scikit-learn의 로드
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

# 재현률, 적합률의 확인
#print(confusion_matrix(Y[test_id], pred_y))
print(recall_score(Y[test_id], pred_y))
print(precision_score(Y[test_id], pred_y))

1.0
0.9225037257824144


#### 모델의 정밀도 향상

In [9]:
# 모델작성용 패러메터를 확인
print(clf)

DecisionTreeClassifier()


In [10]:
# Scikit-learn을 로드
from sklearn.model_selection import GridSearchCV

# 패러메터 범위설정
params = {
    'criterion': ['entropy'],    
    'max_depth': [2, 4, 6, 8, 10],
    'min_samples_leaf': [10, 20, 30, 40, 50],
}

# 그리드 서치의 실행조건을 설정
clf_gs = GridSearchCV(tree.DecisionTreeClassifier(), params, 
                      cv=KFold(n_splits=10, shuffle=True), scoring='accuracy')

# 그리드 서치 실행
clf_gs.fit(X, Y)

GridSearchCV(cv=KFold(n_splits=10, random_state=None, shuffle=True),
             estimator=DecisionTreeClassifier(),
             param_grid={'criterion': ['entropy'],
                         'max_depth': [2, 4, 6, 8, 10],
                         'min_samples_leaf': [10, 20, 30, 40, 50]},
             scoring='accuracy')

In [11]:
# 가장 높은 정밀도와 패터메터의 조합을 표시
print(clf_gs.best_score_)
print(clf_gs.best_params_)

0.8773112319388481
{'criterion': 'entropy', 'max_depth': 10, 'min_samples_leaf': 10}


In [12]:
# 최적의 패터메터를 사용한 모델을 작성
clf_best = tree.DecisionTreeClassifier(
    criterion='entropy', max_depth=10, min_samples_leaf=10)
clf_best.fit(X, Y)

# 영향이 높은 변수를 확인
print(clf_best.feature_importances_)

[0.04715366 0.         0.04932585 0.05899148 0.00677356 0.05967334
 0.44046976 0.03052092 0.06322081 0.01599857 0.00744862 0.00348051
 0.         0.00521295 0.0020835  0.00283342 0.         0.00141202
 0.06973448 0.01432718 0.01466852 0.         0.01801977 0.00255423
 0.0175484  0.00300157 0.00901068 0.03194623 0.01022845 0.01436153
 0.        ]
